In [11]:
import nltk
from nltk.stem.lancaster import LancasterStemmer
import numpy as np
import tensorflow as tf
from tensorflow import keras
import random
import json
import pickle

In [12]:
import numpy as np
import nltk
nltk.download('punkt')
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [13]:
file_path = "/content/intents.json"

with open(file_path, 'r') as file:
    data = json.load(file)

In [14]:

words = []
labels = []
docs_x = []
docs_y = []

for intent in data["intents"]:
	for pattern in intent["patterns"]:
		wrds = nltk.word_tokenize(pattern)
		words.extend(wrds)
		docs_x.append(wrds)
		docs_y.append(intent["tag"])

	if intent["tag"] not in labels:
		labels.append(intent["tag"])

words = [stemmer.stem(w.lower()) for w in words if w != "?"]
words = sorted(list(set(words)))

labels = sorted(labels)


training = []
output = []

out_empty = [0 for _ in range(len(labels))]

for x, doc in enumerate(docs_x):
	bag = []

	wrds = [stemmer.stem(w) for w in doc]

	for w in words:
		if w in wrds:
			bag.append(1)
		else:
			bag.append(0)
	output_row = out_empty[:]
	output_row[labels.index(docs_y[x])] = 1

	training.append(bag)
	output.append(output_row)

training = np.array(training)
output = np.array(output)

In [15]:
# Define the neural network model using TensorFlow 2.0 and Keras
input_size = len(words)
output_size = len(labels)
hidden_size = 8

model = keras.Sequential([
    keras.layers.Dense(hidden_size, input_shape=(input_size,), activation='relu'),
    keras.layers.Dense(output_size, activation='softmax')
])

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [16]:
# Train the model
model.fit(training, output, epochs=1000, batch_size=8)

Epoch 1/1000
24/24 [==============================] - 2s 3ms/step - loss: 3.7870 - accuracy: 0.0213
Epoch 2/1000
24/24 [==============================] - 0s 3ms/step - loss: 3.7652 - accuracy: 0.0213
Epoch 3/1000
24/24 [==============================] - 0s 3ms/step - loss: 3.7477 - accuracy: 0.0266
Epoch 4/1000
24/24 [==============================] - 0s 3ms/step - loss: 3.7273 - accuracy: 0.0479
Epoch 5/1000
24/24 [==============================] - 0s 3ms/step - loss: 3.7047 - accuracy: 0.0638
Epoch 6/1000
24/24 [==============================] - 0s 3ms/step - loss: 3.6786 - accuracy: 0.0798
Epoch 7/1000
24/24 [==============================] - 0s 3ms/step - loss: 3.6507 - accuracy: 0.1064
Epoch 8/1000
24/24 [==============================] - 0s 3ms/step - loss: 3.6188 - accuracy: 0.1330
Epoch 9/1000
24/24 [==============================] - 0s 3ms/step - loss: 3.5846 - accuracy: 0.1383
Epoch 10/1000
24/24 [==============================] - 0s 3ms/step - loss: 3.5468 - accuracy: 0.1277

In [17]:
# Save the model
model.save("model.h5")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [23]:
def bag_of_words(s, words):
    bag = [0] * len(words)

    s_words = nltk.word_tokenize(s)
    s_words = [stemmer.stem(word.lower()) for word in s_words]

    for se in s_words:
        for i, w in enumerate(words):
            if w == se:
                bag[i] = 1

    return np.array(bag)


In [24]:
# Define the chat function
def chat():
    print("Start Talking with the bot (type quit to stop)!")
    while True:
        inp = input("You: ")
        if inp.lower() == "quit":
            break

        # Use the trained model to predict
        results = model.predict(np.array([bag_of_words(inp, words)]))
        results_index = np.argmax(results)
        tag = labels[results_index]

        if results[0, results_index] > 0.5:
            for tg in data["intents"]:
                if tg['tag'] == tag:
                    responses = tg['responses']
            print(random.choice(responses))
            print("\n")
        else:
            print("I didn't get that, try again")

In [25]:
# Load the trained model
model = keras.models.load_model("model.h5")

In [26]:
# Call the chat function
chat()

Start Talking with the bot (type quit to stop)!
You: i have fever
1/1 [==============================] - 0s 128ms/step
To treat a fever at home: 1)Drink plenty of fluids to stay hydrated. 2)Dress in lightweight clothing. 3)Use a light blanket if you feel chilled, until the chills end. 4)Take acetaminophen (Tylenol, others) or ibuprofen (Advil, Motrin IB, others). 5) Get medical help if the fever lasts more than five days in a row.


You: hi
1/1 [==============================] - 0s 16ms/step
I didn't get that, try again
You: i feel pain in my stomach what to do?
1/1 [==============================] - 0s 16ms/step
Rest,Ice,Compression and Elevation can be used to cure strains. Avoid using your muscle for a few days, especially if movement causes an increase in pain and also Apply ice immediately after injuring your muscle. This will minimize swelling. Don’t put ice directly on your skin. Use an ice pack or wrap ice in a towel. To reduce swelling, wrap the affected area with an elastic b

KeyboardInterrupt: Interrupted by user